In [1]:
import pandas as pd
from pyspark.sql.types import StringType
from pyspark.sql.functions import mean, col, udf
from pyspark import SQLContext, SparkContext
import numpy as np
import pandas
import pdb

In [2]:
train_file = 'data/train.csv'
test_file = 'data/test.csv'

In [20]:
sc = SparkContext('local')
sqlContext = SQLContext(sc)

traindata = sqlContext.read.csv(train_file, sep='\t', header=True)
testdata = sqlContext.read.csv(test_file, sep='\t', header=True)

In [21]:
correct = traindata.filter(traindata['Correct First Attempt'] == '1')

In [22]:
# Personal CFAR
student_group = traindata.groupby('Anon Student Id').count()
student_correct_group = correct.groupby('Anon Student Id').count()
student_correct_rate = student_correct_group.join(student_group, student_group['Anon Student Id'] == student_correct_group['Anon Student Id']).drop(
    student_group['Anon Student Id']).select('Anon Student Id', (student_correct_group['count'] / student_group['count']).alias('Personal CFAR'))
student_mean_CFAR = student_correct_rate.select(
    mean(col('Personal CFAR')).alias('mean')).collect()[0]['mean']
traindata = traindata.join(student_correct_rate, student_correct_rate['Anon Student Id'] == traindata['Anon Student Id']).drop(
    student_correct_rate['Anon Student Id'])
testdata = testdata.join(student_correct_rate, student_correct_rate['Anon Student Id'] == testdata['Anon Student Id']).drop(
    student_correct_rate['Anon Student Id'])
testdata.na.fill(student_mean_CFAR, 'Personal CFAR')

DataFrame[Row: string, Anon Student Id: string, Problem Hierarchy: string, Problem Name: string, Problem View: string, Step Name: string, Step Start Time: string, First Transaction Time: string, Correct Transaction Time: string, Step End Time: string, Step Duration (sec): string, Correct Step Duration (sec): string, Error Step Duration (sec): string, Correct First Attempt: string, Incorrects: string, Hints: string, Corrects: string, KC(Default): string, Opportunity(Default): string, Personal CFAR: double]

In [23]:
# Problem CFAR
problem_group = traindata.groupby('Problem Name').count()
problem_correct_group = correct.groupby('Problem Name').count()
problem_correct_rate = problem_correct_group.join(problem_group, problem_group['Problem Name'] == problem_correct_group['Problem Name']).drop(
    problem_group['Problem Name']).select('Problem Name', (problem_correct_group['count'] / problem_group['count']).alias('Problem CFAR'))
problem_mean_CFAR = problem_correct_rate.select(
    mean(col('Problem CFAR')).alias('mean')).collect()[0]['mean']
traindata = traindata.join(problem_correct_rate, problem_correct_rate['Problem Name'] == traindata['Problem Name']).drop(
    problem_correct_rate['Problem Name'])
testdata = testdata.join(problem_correct_rate, problem_correct_rate['Problem Name'] == testdata['Problem Name']).drop(
    problem_correct_rate['Problem Name'])
testdata.na.fill(problem_mean_CFAR, 'Problem CFAR')

DataFrame[Row: string, Anon Student Id: string, Problem Hierarchy: string, Problem Name: string, Problem View: string, Step Name: string, Step Start Time: string, First Transaction Time: string, Correct Transaction Time: string, Step End Time: string, Step Duration (sec): string, Correct Step Duration (sec): string, Error Step Duration (sec): string, Correct First Attempt: string, Incorrects: string, Hints: string, Corrects: string, KC(Default): string, Opportunity(Default): string, Personal CFAR: double, Problem CFAR: double]

In [24]:
# Step CFAR
step_group = traindata.groupby('Step Name').count()
step_correct_group = correct.groupby('Step Name').count()
step_correct_rate = step_correct_group.join(step_group, step_group['Step Name'] == step_correct_group['Step Name']).drop(
    step_group['Step Name']).select('Step Name', (step_correct_group['count'] / step_group['count']).alias('Step CFAR'))
step_mean_CFAR = step_correct_rate.select(
    mean(col('Step CFAR')).alias('mean')).collect()[0]['mean']
traindata = traindata.join(step_correct_rate, step_correct_rate['Step Name'] == traindata['Step Name']).drop(
    step_correct_rate['Step Name'])
testdata = testdata.join(step_correct_rate, step_correct_rate['Step Name'] == testdata['Step Name']).drop(
    step_correct_rate['Step Name'])
testdata.na.fill(step_mean_CFAR, 'Step CFAR')

DataFrame[Row: string, Anon Student Id: string, Problem Hierarchy: string, Problem Name: string, Problem View: string, Step Name: string, Step Start Time: string, First Transaction Time: string, Correct Transaction Time: string, Step End Time: string, Step Duration (sec): string, Correct Step Duration (sec): string, Error Step Duration (sec): string, Correct First Attempt: string, Incorrects: string, Hints: string, Corrects: string, KC(Default): string, Opportunity(Default): string, Personal CFAR: double, Problem CFAR: double, Step CFAR: double]

In [3]:
train_data = pd.read_csv(train_file, sep='\t')
test_data = pd.read_csv(test_file, sep='\t')

In [4]:
compound_KCs = set(train_data['KC(Default)'])
KC ={}
KC_correct_rate = {}
for kcs in compound_KCs:
    if pd.isnull(kcs):
        continue
    for kc in kcs.split('~~'):
        KC[kc] = [0,0]
        
for _, row in train_data.iterrows():
    if pd.isnull(row['KC(Default)']):
        continue
    else:
        for kc in row['KC(Default)'].split('~~'):
            if row['Correct First Attempt']:
                KC[kc][0] += 1
            KC[kc][1] += 1
KC_correct_rate = {i:KC[i][0]/KC[i][1] for i in KC.keys()}
mean_KCFAR =  np.mean(list(KC_correct_rate.values()))

In [5]:
# KC CFAR(avg)
tmp = []
for i, row in train_data.iterrows():
    if i%10000 == 0:
        print(i)
    if pd.isnull(row['KC(Default)']):
        tmp.append(0.815)
    else:
#         for kc in row['KC(Default)'].split('~~'):
        tmp.append(sum([KC_correct_rate[kc] for kc in row['KC(Default)'].split('~~')])\
        / (row['KC(Default)'].count("~")+1))
train_data['KC CFAR(avg)'] = tmp

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000


In [6]:
# KC CFAR
tmp = []
for i, row in train_data.iterrows():
    if i%10000 == 0:
        print(i)
    if pd.isnull(row['KC(Default)']):
        tmp.append(0.815)
    else:
        tmpp = 1.0
        for kc in row['KC(Default)'].split('~~'):
            tmpp *= KC_correct_rate[kc]
        tmp.append(tmpp)
train_data['KC CFAR'] = tmp

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000


In [7]:
# KC_num
train_data['KC_num'] = train_data['KC(Default)'].astype("str").apply(lambda x: 0 if x == 'nan' else (x.count('~~') + 1))

In [8]:
# Opportunity(Mean), Opportunity(Min)
train_data['Opportunity(Mean)'] = train_data['Opportunity(Default)'].astype(
    "str").apply(lambda x: np.mean(list(map(int, x.replace('nan', '0').split('~~')))))
train_data['Opportunity(Min)'] = train_data['Opportunity(Default)'].astype(
    "str").apply(lambda x: min(list(map(int, x.replace('nan', '0').split('~~')))))

In [10]:
# Step time
step_time = {}
for step, group in train_data.groupby(['Step Name']):
    step_time[step] = 1.0 * (group['Step Duration (sec)']).mean() 
                                   
train_data['Step time'] = train_data['Step Name'].apply(lambda x: step_time[x])
tmp_list = []
for i in step_time.values():
    if not pd.isnull(i):
        tmp_list.append(i)
median_time = np.median(tmp_list)
mean_step_time = np.mean(tmp_list)

In [11]:
# Problem time
problem_time = {}
for problem, group in train_data.groupby(['Problem Name']):
    problem_time[problem] = (group['Step Duration (sec)']).mean() 
train_data['Problem time'] = train_data['Problem Name'].apply(
    lambda x: problem_time[x])
mean_prob_time = np.mean(list(problem_time.values()))

In [15]:
# add in test
test_data['KC_num'] = test_data['KC(Default)'].astype("str").apply(
    lambda x: 0 if x == 'nan' else (x.count('~~') + 1))
test_data['Step time'] = test_data['Step Name'].apply(
    lambda x: step_time[x] if x in step_time.keys() else mean_step_time)
test_data['Problem time'] = test_data['Step Name'].apply(
    lambda x: problem_time[x] if x in problem_time.keys() else mean_prob_time)

In [16]:
tmp_list = []
for i, row in test_data.iterrows():
    if pd.isnull(row['KC(Default)']):
        tmp_list.append(0.815) 
    else:
        tmp = 1
        for kc in row['KC(Default)'].split('~~'):
            if kc in KC_correct_rate.keys():
                tmp *= KC_correct_rate[kc]
            else:
                tmp *= mean_KCFAR
        tmp_list.append(tmp)
test_data['KC CFAR'] = tmp_list

tmp_list = []
for i, row in test_data.iterrows():
    if pd.isnull(row['KC(Default)']):
        tmp_list.append(0.815) 
    else:
        tmp = 0
        for kc in row['KC(Default)'].split('~~'):
            if kc in KC_correct_rate.keys():
                tmp += (KC_correct_rate[kc])
            else:
                tmp += (mean_KCFAR)
        tmp_list.append(tmp/(row['KC(Default)'].count("~")+1))
test_data['KC CFAR(avg)'] = tmp_list

In [17]:
test_data['Opportunity(Mean)'] = test_data['Opportunity(Default)'].astype("str").apply(
    lambda x: np.mean(list(map(int, x.replace('nan', '0').split('~~')))))
test_data['Opportunity(Min)'] = test_data['Opportunity(Default)'].astype(
    "str").apply(lambda x: min(list(map(int, x.replace('nan', '0').split('~~')))))

In [18]:
train_data[:3]

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,...,Corrects,KC(Default),Opportunity(Default),KC CFAR(avg),KC CFAR,KC_num,Opportunity(Mean),Opportunity(Min),Step time,Problem time
0,9938,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C1,2005-09-09 12:23:34.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,...,1,NaN,NaN,0.815000,0.815000,0,0.0,0,49.941010,40.733454
1,9939,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C2,2005-09-09 12:24:07.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,...,1,NaN,NaN,0.815000,0.815000,0,0.0,0,32.949725,40.733454
2,9940,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C1,2005-09-09 12:24:22.0,2005-09-09 12:25:16.0,2005-09-09 12:25:40.0,2005-09-09 12:25:40.0,...,1,Define Variable,1,0.966979,0.966979,1,1.0,1,10.225832,40.733454


In [19]:
test_data[:3]

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,...,Corrects,KC(Default),Opportunity(Default),KC_num,Step time,Problem time,KC CFAR,KC CFAR(avg),Opportunity(Mean),Opportunity(Min)
0,10039,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP01,1,2/300*X = 10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,22.200000,24.643487,0.815000,0.815000,0.0,0
1,10482,52vEY7f17k,"Unit CTA1_06, Section CTA1_06-3",FOR05,1,xScale,NaN,NaN,NaN,NaN,...,NaN,Changing axis intervals,28,1,17.450453,24.643487,1.000000,1.000000,28.0,28
2,11024,52vEY7f17k,"Unit ES_03, Section ES_03-6",EG52,1,2.9 = -2x+3.7+x,NaN,NaN,NaN,NaN,...,NaN,"[SkillRule: Consolidate vars, no coeff; CLT]",27,1,21.959627,24.643487,0.896476,0.896476,27.0,27


In [11]:
traindata.columns

['Row',
 'Anon Student Id',
 'Problem Hierarchy',
 'Problem Name',
 'Problem View',
 'Step Name',
 'Step Start Time',
 'First Transaction Time',
 'Correct Transaction Time',
 'Step End Time',
 'Step Duration (sec)',
 'Correct Step Duration (sec)',
 'Error Step Duration (sec)',
 'Correct First Attempt',
 'Incorrects',
 'Hints',
 'Corrects',
 'KC(Default)',
 'Opportunity(Default)',
 'Personal CFAR',
 'Problem CFAR',
 'Step CFAR',
 'KC CFAR']

In [27]:
traindata = traindata.drop('Row')
testdata = testdata.drop('Row')
traindata = traindata.drop('Anon Student Id')
testdata = testdata.drop('Anon Student Id')
traindata = traindata.drop('Problem Hierarchy')
testdata = testdata.drop('Problem Hierarchy')
traindata = traindata.drop('Problem Name')
testdata = testdata.drop('Problem Name')
traindata = traindata.drop('Step Name')
testdata = testdata.drop('Step Name')
traindata = traindata.drop('Step Start Time')
testdata = testdata.drop('Step Start Time')
traindata = traindata.drop('First Transaction Time')
testdata = testdata.drop('First Transaction Time')
traindata = traindata.drop('Correct Transaction Time')
testdata = testdata.drop('Correct Transaction Time')
traindata = traindata.drop('Step End Time')
testdata = testdata.drop('Step End Time')
traindata = traindata.drop('Step Duration (sec)')
testdata = testdata.drop('Step Duration (sec)')
traindata = traindata.drop('Correct Step Duration (sec)')
testdata = testdata.drop('Correct Step Duration (sec)')
traindata = traindata.drop('Error Step Duration (sec)')
testdata = testdata.drop('Error Step Duration (sec)')
traindata = traindata.drop('Incorrects')
testdata = testdata.drop('Incorrects')
traindata = traindata.drop('Hints')
testdata = testdata.drop('Hints')
traindata = traindata.drop('Corrects')
testdata = testdata.drop('Corrects')
traindata = traindata.drop('KC(Default)')
testdata = testdata.drop('KC(Default)')
traindata = traindata.drop('Opportunity(Default)')
testdata = testdata.drop('Opportunity(Default)')

In [28]:
traindata.columns

['Problem View',
 'Correct First Attempt',
 'Personal CFAR',
 'Problem CFAR',
 'Step CFAR']

In [29]:
testdata.columns

['Problem View',
 'Correct First Attempt',
 'Personal CFAR',
 'Problem CFAR',
 'Step CFAR']

In [31]:
train_data['Problem Hierarchy'] = train_data['KC_num']

In [32]:
train = traindata.toPandas()
test = testdata.toPandas()

In [34]:
train['KC_num'] = train_data['KC_num']
test['KC_num'] = test_data['KC_num']
train['KC CFAR'] = train_data['KC CFAR']
test['KC CFAR'] = test_data['KC CFAR']
train['KC CFAR(avg)'] = train_data['KC CFAR(avg)']
test['KC CFAR(avg)'] = test_data['KC CFAR(avg)']
train['Problem time'] = train_data['Problem time']
test['Problem time'] = test_data['Problem time']
train['Step time'] = train_data['Step time']
test['Step time'] = test_data['Step time']
train['Opportunity(Mean)'] = train_data['Opportunity(Mean)']
test['Opportunity(Mean)'] = test_data['Opportunity(Mean)']
train['Opportunity(Min)'] = train_data['Opportunity(Min)']
test['Opportunity(Min)'] = test_data['Opportunity(Min)']

In [35]:
train.head()

,Problem View,Correct First Attempt,Personal CFAR,Problem CFAR,Step CFAR,KC_num,KC CFAR,KC CFAR(avg),Problem time,Step time,Opportunity(Mean),Opportunity(Min)
0,1,1,0.779603,0.889535,1.000000,0,0.815000,0.815000,40.733454,49.941010,0.0,0
1,2,1,0.775149,0.813165,1.000000,0,0.815000,0.815000,40.733454,32.949725,0.0,0
2,2,1,0.788705,0.787606,1.000000,1,0.966979,0.966979,40.733454,10.225832,1.0,1
3,1,1,0.808453,0.779661,1.000000,3,0.124279,0.300044,40.733454,84.677811,1.0,1
4,3,1,0.752012,0.743902,0.666667,1,0.800770,0.800770,40.733454,34.228587,1.0,1


In [36]:
train.to_csv('data/train_pyspark.csv', sep='\t', header=True, index = False)
test.to_csv('data/test_pyspark.csv', sep='\t', header=True, index = False)